In [49]:
import sys
from collections import deque
sys.path.append("/Users/yoavfreund/projects/Webwork.Improvement.Project/AdaptiveHints/src/parsetrees/expr_parser/")
from Eval_parsed import eval_parsed
from webwork_parser import parse_webwork

In [3]:
answer='1 - 1/2^10'
attempt='1-1/2^11.0'


final_pairs=find_matches(answer,attempt)
print final_pairs

attempt list
[0.00048828125, 't', [['/', 0.00048828125, [2, 9]], ['X', 1.0, [2, 2]], [['^', 2048.0, [4, 9]], ['X', 2.0, [4, 4]], ['X', 11.0, [6, 9]]]]]
[0.99951171875, 't', [['-', 0.99951171875, [0, 9]], ['X', 1.0, [0, 0]], [['/', 0.00048828125, [2, 9]], ['X', 1.0, [2, 2]], [['^', 2048.0, [4, 9]], ['X', 2.0, [4, 4]], ['X', 11.0, [6, 9]]]]]]
[1.0, 't', ['X', 1.0, [0, 0]]]
[1.0, 't', ['X', 1.0, [2, 2]]]
[2.0, 't', ['X', 2.0, [4, 4]]]
[11.0, 't', ['X', 11.0, [6, 9]]]
[2048.0, 't', [['^', 2048.0, [4, 9]], ['X', 2.0, [4, 4]], ['X', 11.0, [6, 9]]]]
answer list
[0.0009765625, 'c', [['/', 0.0009765625, [4, 9]], ['X', 1.0, [4, 4]], [['^', 1024.0, [6, 9]], ['X', 2.0, [6, 6]], ['X', 10.0, [8, 9]]]]]
[0.9990234375, 'c', [['-', 0.9990234375, [0, 9]], ['X', 1.0, [0, 0]], [['/', 0.0009765625, [4, 9]], ['X', 1.0, [4, 4]], [['^', 1024.0, [6, 9]], ['X', 2.0, [6, 6]], ['X', 10.0, [8, 9]]]]]]
[1.0, 'c', ['X', 1.0, [0, 0]]]
[1.0, 'c', ['X', 1.0, [4, 4]]]
[2.0, 'c', ['X', 2.0, [6, 6]]]
[10.0, 'c', ['X', 10.

In [1]:
#%%writefile FindMatchingSubexpressions.py
import sys
from collections import deque
sys.path.append("/Users/yoavfreund/projects/Webwork.Improvement.Project/AdaptiveHints/src/parsetrees/expr_parser/")
from Eval_parsed import eval_parsed
from webwork_parser import parse_webwork

def flatten(tree,tag):
    #print 'parsing ',tree
    List=[]
    Queue=deque([tree])
    while Queue:
        current=Queue.popleft()
        if type(current[0])==str:
            List.append([current[1],tag,current])
        elif type(current[0][0])==str and type(current[0][1])==float and type(current[0][2])==list:
            List.append([current[0][1],tag,current])
            Queue.extend(current[1:])
        else:
            print 'parse error:',current
            return None
    List=sorted(List,key=lambda x: x[0])
    return List

def find_Hits(List,tol = 1+1e-6):
    """ Given a combined list of subtrees from both attempt and answer,
    sorted by value, find the matching pairs of trees
    tol is the tolerance used to define which pairs of values match. 
    Needed because different expressions get different roundoff error
    """
    Hits=[]
    item1=List[0]
    for item2 in List[1:]:
        #print item1[:2],item2[:2],
        ratio=item1[0]/item2[0]
        if item1[1]!=item2[1] and ratio <tol and ratio>(1/tol):
            if item1[1]=='c':
                Hits.append((item1,item2))
            elif item2[1]=='c':
                Hits.append((item2,item1))
            else:
                print "Error in find_Hits. Neither item labeled c"
        item1=item2
    return Hits

def get_span(tree):
    if tree[2][0]=='X':
        return tree[2][2]
    elif type(tree[2][0])==list:
        return tree[2][0][2]
    else:
        print 'Error in get_span'
        return None
    
def find_dominating_hits(Hits,answer,attempt):
    for i in range(len(Hits)):
        for j in range(len(Hits)):
            if i==j:
                continue
            span1=get_span(Hits[i][0])
            span2=get_span(Hits[j][0])
            if span1[0]<=span2[0] and span1[1]>=span2[1]:
                print span1,'dominates',span2
                Hits[j][0][1]='dc'
    
    final_pairs=[]
    for hit in Hits:
        if hit[0][1]=='c':
            span_c=get_span(hit[0])
            span_a=get_span(hit[1])
            answer_part=answer[span_c[0]:span_c[1]+1]
            attempt_part = attempt[span_a[0]:span_a[1]+1]
            final_pairs.append((answer_part,attempt_part))
    return final_pairs

def find_matches(answer,attempt):

    attempt_parsed=parse_webwork(attempt)
    attempt_tree=eval_parsed(attempt_parsed)
    attempt_list=flatten(attempt_tree,'t')
    print 'attempt list'
    for item in attempt_list:
        print item
    
    answer_parsed=parse_webwork(answer)
    answer_tree=eval_parsed(answer_parsed)
    answer_list=flatten(answer_tree,'c')
    print 'answer list'
    for item in answer_list:
        print item

    combined_list=sorted(answer_list+attempt_list,key=lambda x: x[0])
    #print 'combined list'
    #for item in combined_list:
    #    print item

    Hits=find_Hits(combined_list)
    i=0
    for item in Hits:
        i+=1
        print i,item

    final_pairs=find_dominating_hits(Hits,answer,attempt)
    #for item in final_pairs:
    #    print "The piece %s in your answer is correct, it can also be expressed as %s"%(item[1],item[0])
    
    return final_pairs

#if __name__=="__main__":
#    final_pairs=find_matches(sys.argv[1],sys.argv[2])
#    for item in final_pairs:
#        print "The piece %s in your answer is correct, it can also be expressed as %s"%(item[1],item[0])

In [35]:
from collections import deque

def flatten(tree,tag):
    #print 'parsing ',tree
    List=[]
    Queue=deque([tree])
    while Queue:
        current=Queue.popleft()
        if type(current[0])==str:
            List.append([current[1],tag,current])
        elif type(current[0][0])==str and type(current[0][1])==float and type(current[0][2])==list:
            List.append([current[0][1],tag,current])
            Queue.extend(current[1:])
        else:
            print 'parse error:',current
            return None
    List=sorted(List,key=lambda x: x[0])
    return List

    answer_list=flatten(answer_tree,'c')
    print 'answer list'
    for item in answer_list:
        print item
    attempt_list=flatten(attempt_tree,'t')
    print 'attempt list'
    for item in attempt_list:
        print item

    combined_list=sorted(answer_list+attempt_list,key=lambda x: x[0])
    print 'combined list'
    for item in combined_list:
        print item


In [36]:
answer_list=flatten(answer_tree,'c')
print 'answer list'
for item in answer_list:
    print item
attempt_list=flatten(attempt_tree,'t')
print 'attempt list'
for item in attempt_list:
    print item

combined_list=sorted(answer_list+attempt_list,key=lambda x: x[0])
print 'combined list'
for item in combined_list:
    print item


answer list
[2.0, 'c', ['X', 2.0, [0, 0]]]
[2.0, 'c', ['X', 2.0, [5, 5]]]
[5.0, 'c', ['X', 5.0, [7, 7]]]
[10.0, 'c', ['X', 10.0, [2, 3]]]
[32.0, 'c', [['^', 32.0, [5, 7]], ['X', 2.0, [5, 5]], ['X', 5.0, [7, 7]]]]
[992.0, 'c', [['-', 992.0, [0, 7]], [['^', 1024.0, [0, 3]], ['X', 2.0, [0, 0]], ['X', 10.0, [2, 3]]], [['^', 32.0, [5, 7]], ['X', 2.0, [5, 5]], ['X', 5.0, [7, 7]]]]]
[1024.0, 'c', [['^', 1024.0, [0, 3]], ['X', 2.0, [0, 0]], ['X', 10.0, [2, 3]]]]
attempt list
[17.0, 't', ['X', 17.0, [5, 6]]]
[1007.0, 't', [['-', 1007.0, [0, 6]], ['X', 1024.0, [0, 3]], ['X', 17.0, [5, 6]]]]
[1024.0, 't', ['X', 1024.0, [0, 3]]]
combined list
[2.0, 'c', ['X', 2.0, [0, 0]]]
[2.0, 'c', ['X', 2.0, [5, 5]]]
[5.0, 'c', ['X', 5.0, [7, 7]]]
[10.0, 'c', ['X', 10.0, [2, 3]]]
[17.0, 't', ['X', 17.0, [5, 6]]]
[32.0, 'c', [['^', 32.0, [5, 7]], ['X', 2.0, [5, 5]], ['X', 5.0, [7, 7]]]]
[992.0, 'c', [['-', 992.0, [0, 7]], [['^', 1024.0, [0, 3]], ['X', 2.0, [0, 0]], ['X', 10.0, [2, 3]]], [['^', 32.0, [5, 7]], ['X

In [37]:
def find_Hits(List,tol = 1+1e-6):
    """ Given a combined list of subtrees from both attempt and answer,
    sorted by value, find the matching pairs of trees
    tol is the tolerance used to define which pairs of values match. 
    Needed because different expressions get different roundoff error
    """
    Hits=[]
    item1=List[0]
    for item2 in List[1:]:
        #print item1[:2],item2[:2],
        ratio=item1[0]/item2[0]
        if item1[1]!=item2[1] and ratio <tol and ratio>(1/tol):
            if item1[1]=='c':
                Hits.append((item1,item2))
            elif item2[1]=='c':
                Hits.append((item2,item1))
            else:
                print "Error in find_Hits. Neither item labeled c"
        item1=item2
    return Hits

In [38]:
Hits=find_Hits(combined_list)
i=0
for item in Hits:
    i+=1
    print i,item

1 ([1024.0, 'c', [['^', 1024.0, [0, 3]], ['X', 2.0, [0, 0]], ['X', 10.0, [2, 3]]]], [1024.0, 't', ['X', 1024.0, [0, 3]]])


In [39]:
def get_span(tree):
    if tree[2][0]=='X':
        return tree[2][2]
    elif type(tree[2][0])==list:
        return tree[2][0][2]
    else:
        print 'Error in get_span'
        return None
    
def find_dominating_hits(Hits,answer,attempt):
    for i in range(len(Hits)):
        for j in range(len(Hits)):
            if i==j:
                continue
            span1=get_span(Hits[i][0])
            span2=get_span(Hits[j][0])
            if span1[0]<=span2[0] and span1[1]>=span2[1]:
                print span1,'dominates',span2
                Hits[j][0][1]='dc'
    
    final_pairs=[]
    for hit in Hits:
        if hit[0][1]=='c':
            span_c=get_span(hit[0])
            span_a=get_span(hit[1])
            answer_part=answer[span_c[0]:span_c[1]+1]
            attempt_part = attempt[span_a[0]:span_a[1]+1]
            final_pairs.append((answer_part,attempt_part))
    return final_pairs

In [40]:
final_pairs=find_dominating_hits(Hits,answer,attempt)
for item in final_pairs:
    print "The piece %s in your answer is correct, it can also be expressed as %s"%(item[1],item[0])

The piece 1024 in your answer is correct, it can also be expressed as 2^10


In [45]:
%%writefile testargv.py
import sys
i=0
for item in sys.argv:
    print i,item
    i+=1
    

Writing testargv.py


In [50]:
!grep def /Users/yoavfreund/projects/Webwork.Improvement.Project/AdaptiveHints/src/parsetrees/expr_parser/Eval_parsed.py

def PrintException():
def is_number(s):
def ncr(n, r):
def find_common_values(e1,e2):
def eval_parsed(e, variables = None):
    def get_number(ev):
def Collect_numbers(etree):
def collection_recursion(T,etree):
def numbers_and_exps(etree, string):
def parse_and_collect_numbers(string):
def parse_and_eval(string, variables=None):
